<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/06-Clasificacion-SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación: SVM

En esta notebook trabajaremos la tarea de clasificación, además, mostraremos el uso del clasificador **SVM** (Support Vector Machine). Realizaremos un ejemplo con datos artificiales, con fines didácticos.

Nos enfocaremos en los siguientes puntos:
*   Uso de kernels.
*   Efecto del parámetro $C$.
*   Busqueda de hiperparámetros.
*   Balanceo de clases.

Usaremos la implementación de sklearn, llamada [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) (Support Vector Classifier)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#@title Funciones que necesitamos para graficar las fronteras de decisión

def make_meshgrid(x, y, h=.02):
    '''
    función para hacer la malla de puntos para colorear las regiones de decisión,
    la malla de puntos abarca la región donde se encuentran los puntos (x,y)
    'h' es el tamaño de paso
    '''
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

**Los primeros dos ejemplos son datasets artificiales, con fines ilustrativos**

## Ejemplo 1: Un caso linealmente separable

En este primer ejemplo mostramos el uso básico del SVM como clasificador en un ejemplo ilustrativo con datos linealmente separables.

### El conjunto de datos

Usaremos la función [`make_blobs`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html) que genera conjuntos de datos sintéticos formados por grupos (blobs) de puntos normalmente distribuidos.

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=200,
                  n_features=2,
                  centers=2,
                  random_state=57)

print(X.shape, y.shape)

plt.figure()
plt.scatter(X[:,0], X[:,1],
            c=y # Observar que el color lo especificamos con el arreglo de etiquetas
            )
plt.show()

Veamos el vector de etiquetas

In [ ]:
y

Veamos el balance entre etiquetas

In [ ]:
print(f"Instancias de la clase positiva: {y[y==1].shape[0]}. Porcentaje: {round(100*y[y==1].shape[0]/y.shape[0],2)}%")
print(f"Instancias de la clase negativa: {y[y==0].shape[0]}. Porcentaje: {round(100*y[y==0].shape[0]/y.shape[0],2)}%")

![ML proceso](https://drive.google.com/uc?id=1S9KVyZbkiciIEeC7cLi-epa62gfFM0pi)


### La división *train/test*

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

print(f"X Train: {X_train.shape}")
print(f"X Test: {X_test.shape}")
print(f"Y Train: {y_train.shape}")
print(f"Y Test: {y_test.shape}")

### Entrenamiento

Entrenemos y evaluemos el clasificador en el conjunto de entrenamiento

In [ ]:
from sklearn.svm import SVC

#---- inicializar la clase ----
clf = SVC(kernel='linear') # Sin hiperparametros se tiene C=1

#---- entrenar el modelo ----
clf.fit(X_train, y_train)

#---- evaluar el modelo ----
score = clf.score(X_train,y_train)  # Este score es el accuracy
print(f"Accuracy promedio de entrenamiento: {score}")

### Evaluación

Evaluemos varias métricas en el conjunto de prueba

In [ ]:
clf.score(X_test,y_test)    # Este score es el accuracy

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, confusion_matrix

y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test,y_pred)
print(cm)

plt.figure()
plt.imshow(cm)
plt.colorbar()
plt.xticks([0,1])
plt.yticks([0,1])
plt.xlabel("Etiqueta predicha")
plt.ylabel("Etiqueta real")
plt.show()

In [ ]:
print(f"Accuracy: {accuracy_score(y_test,y_pred)}")
print(f"Recall: {recall_score(y_test,y_pred)}")
print(f"Precision: {precision_score(y_test,y_pred)}")
print(f"F1 Score: {f1_score(y_test,y_pred)}")

Obtuvimos métricas de rendimiento del 100%, **¿te parece sospechoso?**

In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

Algunos hiperparámetros:

* `C`:  Este parámetro es fundamental para encontrar un equilibrio adecuado entre el ajuste del modelo a los datos de entrenamiento y la capacidad del modelo para generalizar a datos nuevos.
* `kernel`: Tipo de kernel usado.


Algunos métodos:

*   `score()`: Accuracy del clasificador en el conjunto especificado.
*   `predict()`: Predecir la clase del conjunto especificado usando el modelo ya entrenado.



En este ejemplo, ¿qué efecto tiene el parámetro $C$?

In [ ]:
from sklearn.svm import SVC

parametro_c = 0.1

clf_c = SVC(C=parametro_c,kernel='linear') # Sin hiperparametros se tiene C=1
clf_c.fit(X_train, y_train)
clf_c.score(X_train,y_train)  # Este score es el accuracy

xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones
fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")
plot_contours(ax1, clf_c, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')
plot_contours(ax2, clf_c, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

In [ ]:
clf_c.score(X_train,y_train)

## Ejemplo 2: Un caso no linealmente separable

Ahora, analicemos otro ejemplo *toy* que no sea linealmente separable. Usaremos direntes kernels

### El conjunto de datos

Creamos un conjunto de datos con una condición XOR

In [ ]:
# @title
np.random.seed(17) # Fijamos un seed para la reproducibilidad de resultados

X = np.random.randn(500, 2)
y = np.array([int(np.logical_xor(x[0] > 0, x[1] > 0)) for x in X])  # las etiquetas las asignamos en función de la comparación x0>0 XOR x1>0

plt.figure()
plt.scatter(X[y==0, 0], X[y==0, 1], color='blue', label='Clase 0')
plt.scatter(X[y==1, 0], X[y==1, 1], color='red',label='Clase 1')
plt.legend()
plt.show()

Veamos el balance entre etiquetas

In [ ]:
print(f"Instancias de la clase positiva: {y[y==1].shape[0]}. Porcentaje: {round(100*y[y==1].shape[0]/y.shape[0],2)}%")
print(f"Instancias de la clase negativa: {y[y==0].shape[0]}. Porcentaje: {round(100*y[y==0].shape[0]/y.shape[0],2)}%")

### Separación *train/test*

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023)

print(f"X Train: {X_train.shape}")
print(f"X Test: {X_test.shape}")
print(f"Y Train: {y_train.shape}")
print(f"Y Test: {y_test.shape}")

### Clasificación

#### Kernel Lineal

Entrenemos el clasificador usando el kernel lineal. Observar que, por default, $C=1$.

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

Observemos el accuracy en el conjunto de entrenamiento y prueba. En este caso, el método `score` de la clase `SVC` calcula el accuracy.

In [ ]:
print(f"Accuracy promedio de entrenamiento: {round(clf.score(X_train, y_train),3)}")
print(f"Accuracy promedio de prueba: {round(clf.score(X_test, y_test),3)}")

Observemos la frontera de decisión calculada por el clasificador y los conjuntos de entrenamiento y prueba.

In [ ]:
#@title Graficar las fronteras de decisión

xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear',C=0.005)
clf.fit(X_train, y_train)

print(f"Accuracy promedio de entrenamiento: {round(clf.score(X_train, y_train),3)}")
print(f"Accuracy promedio de prueba: {round(clf.score(X_test, y_test),3)}")

In [ ]:
#@title Graficar las fronteras de decisión

xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

#### Kernel Polinomial

Dado que los datos no son linealmente separables, usemos un kernel no lineal.

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='poly',gamma=2,degree=2,C=1)
clf.fit(X_train, y_train)

print(f"Accuracy promedio de entrenamiento: {round(clf.score(X_train, y_train),3)}")
print(f"Accuracy promedio de prueba: {round(clf.score(X_test, y_test),3)}")

In [ ]:
#@title Graficar las fronteras de decisión

xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

### ⭕ Ejercicio

Probemos otros kernels. Tenemos disponibles varios kernels.

Con los mismos conjuntos de prueba y entrenamiento y con $C=1$:

1. Repetir el experimento de clasificación de arriba, usando otros kernels.
2. En cada caso que pruebes grafica los puntos (los de prueba) y la frontera de decisión.
3. En cada caso, reporta el valor de accuracy y F1 score, usando el conjunto de prueba solamente.

**¿Qué kernel parece dar mejores métricas de rendimiento?**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023)

'''
------------------------
COMPLETA EL CÓDIGO:
* Usa un kernel que no sea lineal con C=1
* Obten el accuracy en el conjunto de entrenamiento
* Genera las predicciones sobre el conjunto de prueba
* Reporta las métricas Accuracy y F1-score en el conjunto de prueba
* Grafica las fronteras de decisión
------------------------
'''

clf =



In [ ]:
#@title Graficar las fronteras de decisión

xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

* El kernel lineal es mejor para datos linealmente separables. Es una opción cuando el conjunto de datos es grande.
* El kernel Gaussiano (RBF) tiende a dar buenos resultados cuando no se tiene información adicional sobre los datos.
* Los kernels polinomiales tienden a dar buenos resultados cuando los datos de entrenamiento están normalizados.

[El truco del kernel](https://www.geogebra.org/m/xawkavxe)

⭕ Prueba también con otros valores de `C` y repite los pasos de arriba, ¿qué efecto tiene el modificar este valor en la clasificación y en la frontera de decisión?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023)

'''
------------------------
COMPLETA EL CÓDIGO:
* Usa un kernel Gaussiano con diferentes valores de C
* Obten el accuracy en el conjunto de entrenamiento
* Genera las predicciones sobre el conjunto de prueba
* Reporta las métricas Accuracy y F1-score en el conjunto de prueba
* Grafica las fronteras de decisión
------------------------
'''

clf =

In [ ]:
#@title Graficar las fronteras de decisión

xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

### ⚡ Usando gridsearch para encontrar los mejores parámetros

[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) toma un estimador (por ejemplo, SVM) y un conjunto de parámetros del estimador. Sobre estos parámetros hace una busqueda para encontrar la combinación de parámetros que da mejores resultados en el estimador.

GridSearchCV tiene métodos “fit” y “score” method, entre otros. Es decir, no es necesario tomar los parámetros e introducirlos en el estimador.

In [ ]:
from sklearn.model_selection import GridSearchCV

Encuentra los mejores parámetros para el clasificador SVM utilizando grid search. Guíate por el desempeño en el set de entrenamiento y validación.

Prueba los siguientes hyperparámetros.
* kernel = linear, polynomial, rbf
* C = 0.01, 0.1, 1.0, 10, 100
* grado del polinomio = 1, 2, 3, 4 (solo para el kernel polinomial)
* gamma = auto, scale:

Definimos los parámetros sobre los que se hará la busqueda

In [ ]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ('linear', 'poly', 'rbf'),
              'degree': [2, 3, 4], 'gamma': ('auto', 'scale')}
param_grid

Realizamos una busqueda sobre estos parámetros

In [ ]:
clf = SVC()
gs = GridSearchCV(clf, param_grid)
gs.fit(X_train, y_train)

Veamos los mejores hiper-parámetros

In [ ]:
print(f"Best score: {gs.best_score_:.4f}")
print(f"Best params: {gs.best_params_}")

Definamos un clasificador SVM con estos mejores hiperparámetros

In [ ]:
best_svm = SVC(C=100, kernel='poly', degree=2, gamma='auto')
best_svm.fit(X_train, y_train)

print(f"Train mean accuracy: {best_svm.score(X_train, y_train):6.4f}")
print(f"Test mean accuracy: {best_svm.score(X_test, y_test):6.4f}")

In [ ]:
best_clf = gs.best_estimator_

In [ ]:
best_clf.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = best_clf.predict(X_test)

confusion_matrix(y_test,y_pred)

Graficamos la frontera de decisión

In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, ax = plt.subplots(dpi=100)  # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen
plot_contours(ax, best_svm, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title('Frontera de decisión del SVM')
plt.show()

## Ejemplo 3: Clasificación Binaria

Ahora veamos un ejemplo con un dataset real.

- **Contenido**: Clasificación de tumores de mama (malignos o benignos)
- **Tamaño**: 569 muestras con 30 características
- **Target**: Binario (0: benigno, 1: maligno)
- **Origen**: Repositorio UCI Machine Learning, Dr. William H. Wolberg (Universidad de Wisconsin)


Las características provienen de imágenes digitalizadas de aspiraciones con aguja fina (FNA) de masas mamarias e incluyen:

- Radio
- Textura
- Perímetro
- Área
- Suavidad
- Compacidad
- Concavidad
- Puntos cóncavos
- Simetría
- Dimensión fractal

Para cada característica, se incluyen:
- Valor medio
- Error estándar
- "Peor" valor (media de los tres mayores valores)


[Más información](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)

In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

data = load_breast_cancer()
X, y = data.data, data.target

# Features clave (5 seleccionadas por simplicidad):
# 1. 'mean radius' (tamaño del núcleo celular)
# 2. 'mean texture' (variación de color)
# 3. 'mean perimeter'
# 4. 'mean smoothness' (suavidad de bordes)
# 5. 'mean compactness' (circularidad)

# Target binario:
# 0 = maligno, 1 = benigno (balance: 60%-40% aprox.)
cols = ['mean radius', 'mean texture', 'mean perimeter', 'mean smoothness', 'mean compactness']
df = pd.DataFrame(X, columns=data.feature_names)[cols]
df['target'] = y
df

### Análisis Exploratorio

¿Hay datos faltantes?

In [ ]:
df.isnull().sum()

En un problema de clasificación es muy importante ver el balance de las clases

In [ ]:
from seaborn import countplot

labels = np.unique(y)
counts = df['target'].value_counts()

plt.figure()
plt.bar(labels, counts)
plt.xticks(labels)
plt.title("Distribución de clases")
plt.show()

In [ ]:
df.describe()

### Impacto de la multicolinealidad en clasificación

**Modelos afectados**:

- **Modelos lineales**: Inestabilidad en coeficientes (Regresión Logística, SVM lineal).  
- **Optimización con L2**: Pesos distorsionados (redes neuronales).  
- **Modelos sensibles a escala**: Dominio de features redundantes (KNN, SVM-RBF).  

**Modelos no afectados**
- **Árboles y ensembles**: Random Forest, XGBoost (seleccionan features por importancia).  
- **Reducción de dimensionalidad**: PCA, t-SNE (manejan correlaciones automáticamente).  

**Acciones recomendadas**  
1. Eliminar features redundantes (matriz de correlación, VIF).  
2. Selección de features (SelectKBest, RFE).  
3. Regularización (L1/Lasso para eliminar features).  
4. PCA si la interpretabilidad no es prioritaria.  

En resumen, la ML es crítica en modelos lineales o interpretativos. Es menos relevante en árboles o métodos no lineales. Hay que priorizar según objetivo: explicabilidad vs rendimiento.  

In [ ]:
from seaborn import pairplot
import matplotlib.pyplot as plt

plt.figure()
pairplot(df, hue='target')
plt.show()

In [ ]:
df.iloc[:,:-1].corr()

In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

print(f"X: {X.shape}")
print(f"y: {y.shape}")

### Entrenamiento de modelos de clasificación

Entrenemos un primero modelo *baseline*. Sin preprocesamiento.

⭕ Completa el código

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#--- Dividimos en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2025)

#--- Inicializa una clase de SVC ---


#--- Entrena el clasificador ---


#--- Evalua en el conjunto de prueba ---
y_pred = ...
accuracy = ...
f1 = ...
print(f"Accuracy: {accuracy}")
print(f"F1-score: {f1}")

Ahora probemos el efecto de diferentes combinaciones de hiperparámetros usando un pipeline. Probemos:

* Quitar algunas variables, por ejemplo, las correlacionadas.
* Hacer reescalamiento.
* Probar varios valores de $C$ y *kernel*.

Además, reflexionemos sobre lo siguiente:

En diagnóstico médico, se prioriza la **sensibilidad (recall)** para minimizar falsos negativos, ya que es más grave no detectar una enfermedad (ej: cáncer) que tener falsas alarmas. Complementariamente, se analiza el **F1-score** para balancear precisión y sensibilidad, junto con curvas **ROC-AUC** para evaluar el rendimiento general del modelo ante distintos umbrales de decisión.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt


#---- Usando todas las features ---
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

#---- Escogiendo algunas variables ---
# X = df[['mean radius', 	'mean texture', 'mean smoothness', 'mean compactness']].values
# y = df.iloc[:,-1].values

#--- Dividimos en train/test ---
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25, random_state=2025)

pl = Pipeline([
            ('scaler', MinMaxScaler()),
            ('clf', SVC(C=3,kernel='linear'))])

pl.fit(X_train, y_train)

y_pred = pl.predict(X_test)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

cm = confusion_matrix(y_test, y_pred)
plt.figure()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
plt.xticks([0, 1], ['Benigno', 'Maligno'])
plt.yticks([0, 1], ['Benigno', 'Maligno'])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

### Gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10, 100],
              'kernel': ['linear','rbf'],
              'gamma': ['auto','scale']}

clf = SVC()
gs = GridSearchCV(clf, param_grid)
gs.fit(X_train, y_train)

print(f"Best score: {gs.best_score_:.4f}")
print(f"Best params: {gs.best_params_}")

⚡ Retomemos el concepto de overfitting

In [ ]:
cs = np.logspace(-1,1.7,8)
accs_train = []
accs_test = []

for c in cs:
    pl = Pipeline([
            ('scaler', MinMaxScaler()),
            ('clf', SVC(C=c,kernel='rbf'))])
    pl.fit(X_train, y_train)
    y_pred = pl.predict(X_test)
    accs_test.append(accuracy_score(y_test, y_pred))
    y_pred = pl.predict(X_train)
    accs_train.append(accuracy_score(y_train, y_pred))

plt.figure()
plt.suptitle('C vs Accuracy')
plt.plot(cs, accs_train, label='Train')
plt.plot(cs, accs_test, label='Test')
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Ejemplo 4: MNIST

Ahora, trabajemos un ejemplo real, con dimensionalidad alta y no linealmente separable.

### El conjunto de datos

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

X = digits.data
y = digits.target

print(X.shape, y.shape)

Veamos un para de instancias, con sus respectivas etiquetas

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.subplot(1,2,1)
plt.imshow(X[0].reshape(8,8),cmap='gray')
plt.title(y[0])
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(X[1].reshape(8,8),cmap='gray')
plt.title(y[1])
plt.axis('off')
plt.show()

Recordar que cada instancia es un vector de 64 entradas:

In [ ]:
X[0]

Usando [numpy.unique()](https://numpy.org/doc/stable/reference/generated/numpy.unique.html), veamos el balanceo de las clases:

In [ ]:
y[150]

In [ ]:
u, c = np.unique(y, return_counts=True)

print(f"Etiquetas:\n{u}\n")
print(f"Conteos de ocurrencia de cada etiqueta:\n{c}\n")

In [ ]:
plt.figure()
plt.bar(u,c)
plt.xticks(u)
plt.show()

### Separación train/test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=99,train_size=0.8)

print(X_train.shape)
print(X_test.shape)

### Clasificación y evaluación

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC()
clf.fit(X_train,y_train)
y_train_pred = clf.predict(X_train)
y_pred = clf.predict(X_test)
print(f"Train Accuracy: {accuracy_score(y_train,y_train_pred)}")
print(f"Test Accuracy: {accuracy_score(y_test,y_pred)}")

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)
confusion_matrix(y_test,y_pred)

### ⭕ Ejercicio


1. Haz una busqueda de hiperparámetros con GridSearchCV sobre los siguientes parámetros:

    *   C: 0.1,1,10,100,1000
    *   Kernel: lineal, polinomial (grados 3,5,7), Gaussiano ($\gamma$ `auto`, 0.1, 1, 10).
    *   `coef_0`: 0, 1, 2.

2. Usando los mejores parámetros del `GridSearchCV` define un clasificador. Realiza las predicciones sobre el conjunto de prueba y evalua las predicciones usando las métricas accuracy, recall y la matriz de confusión.



Reflexiona sobre las siguientes preguntas:

1. ¿Por qué el accuracy en el conjunto de entrenamiento no es el mismo que el `best_score_`?
2. ¿Por qué el accuracy en el conjunto de prueba no es el mismo que el `best_score_`?
3. ¿Qué digitos son los que se están *confundiendo*?